# Deep cortical reinforcement learning: Policy gradients + Q-learning + GAN


In this notebook, we'll build a neural network that can learn to play games through reinforcement learning. More specifically, we'll use Q-learning to train an agent to play a game called [Cart-Pole](https://gym.openai.com/envs/CartPole-v0). In this game, a freely swinging pole is attached to a cart. The cart can move to the left and right, and the goal is to keep the pole upright as long as possible.

![Cart-Pole](assets/cart-pole.jpg)

We can simulate this game using [OpenAI Gym](https://gym.openai.com/). First, let's check out how OpenAI Gym works. Then, we'll get into training an agent to play the Cart-Pole game.

In [1]:
# In this one we should define and detect GPUs for tensorflow
# GPUs or CPU
import tensorflow as tf

# Check TensorFlow Version
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


>**Note:** Make sure you have OpenAI Gym cloned into the same directory with this notebook. I've included `gym` as a submodule, so you can run `git submodule --init --recursive` to pull the contents into the `gym` repo.

##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import gym

## Create the Cart-Pole game environment
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


/home/arasdar/anaconda3/envs/env/lib/python3.6/site-packages/gym/__init__.py:22: UserWarning: DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.
  warnings.warn('DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.')


We interact with the simulation through `env`. To show the simulation running, you can use `env.render()` to render one frame. Passing in an action as an integer to `env.step` will generate the next step in the simulation.  You can see how many actions are possible from `env.action_space` and to get a random action you can use `env.action_space.sample()`. This is general to all Gym games. In the Cart-Pole game, there are two possible actions, moving the cart left or right. So there are two actions we can take, encoded as 0 and 1.

Run the code below to watch the simulation run.

In [3]:
env.reset()
batch = []
for _ in range(1111):
    #env.render()
    action = env.action_space.sample()
    state, reward, done, info = env.step(action) # take a random action
    batch.append([action, state, reward, done, info])
    #print('state, action, reward, done, info:', state, action, reward, done, info)
    if done:
        env.reset()

To shut the window showing the simulation, use `env.close()`.

In [4]:
# env.close()

If you ran the simulation above, we can look at the rewards:

In [5]:
batch[0], 
batch[0][1].shape, state.shape

((4,), (4,))

In [6]:
import numpy as np
actions = np.array([each[0] for each in batch])
states = np.array([each[1] for each in batch])
rewards = np.array([each[2] for each in batch])
dones = np.array([each[3] for each in batch])
infos = np.array([each[4] for each in batch])

In [7]:
# print(rewards[-20:])
print('shapes:', np.array(rewards).shape, np.array(states).shape, np.array(actions).shape, np.array(dones).shape)
print('dtypes:', np.array(rewards).dtype, np.array(states).dtype, np.array(actions).dtype, np.array(dones).dtype)
print('states:', np.max(np.array(states)), np.min(np.array(states)))
print('actions:', np.max(np.array(actions)), np.min(np.array(actions)))
# print((np.max(np.array(actions)) - np.min(np.array(actions)))+1)
print('rewards:', np.max(np.array(rewards)), np.min(np.array(rewards)))

shapes: (1111,) (1111, 4) (1111,) (1111,)
dtypes: float64 float64 int64 bool
states: 2.208870795028335 -2.5781655146810603
actions: 1 0
rewards: 1.0 1.0


In [8]:
actions[:10]

array([0, 1, 1, 0, 1, 1, 1, 1, 1, 1])

In [9]:
rewards[:10]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [10]:
# import numpy as np
def sigmoid(x, derivative=False):
  return x*(1-x) if derivative else 1/(1+np.exp(-x))

In [11]:
sigmoid(np.max(np.array(rewards))), sigmoid(np.min(np.array(rewards)))

(0.7310585786300049, 0.7310585786300049)

In [12]:
print('rewards:', np.max(np.array(rewards))/100, np.min(np.array(rewards))/100)

rewards: 0.01 0.01


The game resets after the pole has fallen past a certain angle. For each frame while the simulation is running, it returns a reward of 1.0. The longer the game runs, the more reward we get. Then, our network's goal is to maximize the reward by keeping the pole vertical. It will do this by moving the cart to the left and the right.

## Q-Network

We train our Q-learning agent using the Bellman Equation:

$$
Q(s, a) = r + \gamma \max{Q(s', a')}
$$

where $s$ is a state, $a$ is an action, and $s'$ is the next state from state $s$ and action $a$.

Before we used this equation to learn values for a Q-_table_. However, for this game there are a huge number of states available. The state has four values: the position and velocity of the cart, and the position and velocity of the pole. These are all real-valued numbers, so ignoring floating point precisions, you practically have infinite states. Instead of using a table then, we'll replace it with a neural network that will approximate the Q-table lookup function.

<img src="assets/deep-q-learning.png" width=450px>

Now, our Q value, $Q(s, a)$ is calculated by passing in a state to the network. The output will be Q-values for each available action, with fully connected hidden layers.

<img src="assets/q-network.png" width=550px>


As I showed before, we can define our targets for training as $\hat{Q}(s,a) = r + \gamma \max{Q(s', a')}$. Then we update the weights by minimizing $(\hat{Q}(s,a) - Q(s,a))^2$. 

For this Cart-Pole game, we have four inputs, one for each value in the state, and two outputs, one for each action. To get $\hat{Q}$, we'll first choose an action, then simulate the game using that action. This will get us the next state, $s'$, and the reward. With that, we can calculate $\hat{Q}$ then pass it back into the $Q$ network to run the optimizer and update the weights.

Below is my implementation of the Q-network. I used two fully connected layers with ReLU activations. Two seems to be good enough, three might be better. Feel free to try it out.

In [34]:
# Data of the model
def model_input(state_size, lstm_size, batch_size=1):
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    targetQs = tf.placeholder(tf.float32, [None], name='targetQs')
    # GRU: Gated Recurrent Units
    gru = tf.nn.rnn_cell.GRUCell(lstm_size) # hidden size
    cell = tf.nn.rnn_cell.MultiRNNCell([gru], state_is_tuple=False)
    g_initial_state = cell.zero_state(batch_size, tf.float32) # feedback or lateral/recurrent connection from output
    d_initial_state = cell.zero_state(batch_size, tf.float32) # feedback or lateral/recurrent connection from output
    return states, actions, targetQs, cell, g_initial_state, d_initial_state

In [35]:
# How to use batch-norm
#   x_norm = tf.layers.batch_normalization(x, training=training)

#   # ...

#   update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
#   with tf.control_dependencies(update_ops):
#     train_op = optimizer.minimize(loss)

In [36]:
# training: Either a Python boolean, or a TensorFlow boolean scalar tensor (e.g. a placeholder). 
# Whether to return the output in: 
# training mode (normalized with statistics of the current batch) or 
# inference mode (normalized with moving statistics). 
# NOTE: make sure to set this parameter correctly, or else your training/inference will not work properly.

In [37]:
# MLP & Conv
# # Generator/Controller: Generating/prediting the actions
# def generator(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
#     with tf.variable_scope('generator', reuse=reuse):
#         # First fully connected layer
#         h1 = tf.layers.dense(inputs=states, units=hidden_size)
#         bn1 = tf.layers.batch_normalization(h1, training=training)        
#         nl1 = tf.maximum(alpha * bn1, bn1)
        
#         # Second fully connected layer
#         h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
#         bn2 = tf.layers.batch_normalization(h2, training=training)        
#         nl2 = tf.maximum(alpha * bn2, bn2)
        
#         # Output layer
#         logits = tf.layers.dense(inputs=nl2, units=action_size)        
#         #predictions = tf.nn.softmax(logits)

#         # return actions logits
#         return logits

In [38]:
# RNN generator or sequence generator
def generator(states, initial_state, cell, lstm_size, num_classes, reuse=False): 
    with tf.variable_scope('generator', reuse=reuse):
        # First fully connected layer
        inputs = tf.layers.dense(inputs=states, units=lstm_size)
        print(states.shape, inputs.shape)
        
        # with tf.variable_scope('dynamic_rnn_', reuse=tf.AUTO_REUSE):
        # dynamic means adapt to the batch_size
        inputs_rnn = tf.reshape(inputs, [1, -1, lstm_size]) # NxH -> 1xNxH
        print(inputs_rnn.shape, initial_state.shape)
        outputs_rnn, final_state = tf.nn.dynamic_rnn(cell=cell, inputs=inputs_rnn, initial_state=initial_state)
        print(outputs_rnn.shape, final_state.shape)
        outputs = tf.reshape(outputs_rnn, [-1, lstm_size]) # 1xNxH -> NxH
        print(outputs.shape)

        # Last fully connected layer
        logits = tf.layers.dense(inputs=outputs, units=num_classes)
        print(logits.shape)
        #predictions = tf.nn.softmax(logits)
        
        # logits are the action logits
        return logits, final_state

In [39]:
# MLP & Conv
# # Discriminator/Dopamine: Reward function/planner/naviator/advisor/supervisor/cortical columns
# def discriminator(states, actions, hidden_size, reuse=False, alpha=0.1, training=False):
#     with tf.variable_scope('discriminator', reuse=reuse):
#         # Fusion/merge states and actions/ SA/ SM
#         x_fused = tf.concat(axis=1, values=[states, actions])
        
#         # First fully connected layer
#         h1 = tf.layers.dense(inputs=x_fused, units=hidden_size)
#         bn1 = tf.layers.batch_normalization(h1, training=training)        
#         nl1 = tf.maximum(alpha * bn1, bn1)
        
#         # Second fully connected layer
#         h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
#         bn2 = tf.layers.batch_normalization(h2, training=training)        
#         nl2 = tf.maximum(alpha * bn2, bn2)
        
#         # Output layer
#         logits = tf.layers.dense(inputs=nl2, units=1)        
#         #predictions = tf.nn.softmax(logits)

#         # return rewards logits
#         return logits

In [40]:
# RNN generator or sequence generator
def discriminator(states, actions, initial_state, cell, lstm_size, reuse=False): 
    with tf.variable_scope('discriminator', reuse=reuse):
        # Fusion/merge states and actions/ SA/ SM
        x_fused = tf.concat(axis=1, values=[states, actions])
        
        # First fully connected layer
        inputs = tf.layers.dense(inputs=x_fused, units=lstm_size)
        print(states.shape, inputs.shape)
        
        # with tf.variable_scope('dynamic_rnn_', reuse=tf.AUTO_REUSE):
        inputs_rnn = tf.reshape(inputs, [1, -1, lstm_size]) # NxH -> 1xNxH
        print(inputs_rnn.shape, initial_state.shape)
        outputs_rnn, final_state = tf.nn.dynamic_rnn(cell=cell, inputs=inputs_rnn, initial_state=initial_state)
        print(outputs_rnn.shape, final_state.shape)
        outputs = tf.reshape(outputs_rnn, [-1, lstm_size]) # 1xNxH -> NxH
        print(outputs.shape)

        # Last fully connected layer
        logits = tf.layers.dense(inputs=outputs, units=1)
        print(logits.shape)
        #predictions = tf.nn.softmax(logits)
        
        # logits are the action logits
        return logits, final_state

In [41]:
def model_loss(action_size, hidden_size, states, actions, targetQs,
               cell, g_initial_state, d_initial_state):
    # G/Actor
    #actions_logits = generator(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_logits, g_final_state = generator(states=states, num_classes=action_size, 
                                              cell=cell, initial_state=g_initial_state, lstm_size=hidden_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    neg_log_prob_actions = tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels)
    g_loss = tf.reduce_mean(neg_log_prob_actions * targetQs)
    
    # D/Critic
    #Qs_logits = discriminator(actions=actions_logits, hidden_size=hidden_size, states=states)
    Qs_logits, d_final_state = discriminator(states=states, actions=actions_logits, 
                                             cell=cell, initial_state=d_initial_state, lstm_size=hidden_size)
    d_lossQ = tf.reduce_mean(tf.square(tf.reshape(Qs_logits, [-1]) - targetQs))
    d_lossQ_sigm = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=tf.reshape(Qs_logits, [-1]),
                                                                          labels=tf.nn.sigmoid(targetQs)))
    d_loss = d_lossQ_sigm + d_lossQ

    return actions_logits, Qs_logits, g_final_state, d_final_state, g_loss, d_loss, d_lossQ, d_lossQ_sigm

In [42]:
# Optimizating/training/learning G & D
def model_opt(g_loss, d_loss, learning_rate):
    """
    Get optimization operations in order
    :param g_loss: Generator loss Tensor for action prediction
    :param d_loss: Discriminator loss Tensor for reward prediction for generated/prob/logits action
    :param learning_rate: Learning Rate Placeholder
    :return: A tuple of (qfunction training, generator training, discriminator training)
    """
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    g_vars = [var for var in t_vars if var.name.startswith('generator')]
    d_vars = [var for var in t_vars if var.name.startswith('discriminator')]

    # Optimize RNN
    # g_grads, _ = tf.clip_by_global_norm(t_list=tf.gradients(g_loss, g_vars), clip_norm=5) # usually around 1-5
    # d_grads, _ = tf.clip_by_global_norm(t_list=tf.gradients(d_loss, d_vars), clip_norm=5) # usually around 1-5
    g_grads=tf.gradients(g_loss, g_vars)
    d_grads=tf.gradients(d_loss, d_vars)
    g_opt = tf.train.AdamOptimizer(learning_rate).apply_gradients(grads_and_vars=zip(g_grads, g_vars))
    d_opt = tf.train.AdamOptimizer(learning_rate).apply_gradients(grads_and_vars=zip(d_grads, d_vars))
    
    # # Optimize MLP & CNN
    # with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
    #     g_opt = tf.train.AdamOptimizer(learning_rate).minimize(g_loss, var_list=g_vars)
    #     d_opt = tf.train.AdamOptimizer(learning_rate).minimize(d_loss, var_list=d_vars)

    return g_opt, d_opt

In [43]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, learning_rate):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.targetQs, cell, self.g_initial_state, self.d_initial_state = model_input(
            state_size=state_size, lstm_size=hidden_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.Qs_logits, self.g_final_state, self.d_final_state, self.g_loss, self.d_loss, self.d_lossQ, self.d_lossQ_sigm = model_loss(
            action_size=action_size, hidden_size=hidden_size,
            states=self.states, actions=self.actions, cell=cell, targetQs=self.targetQs,
            g_initial_state=self.g_initial_state, d_initial_state=self.d_initial_state)
        
        # Update the model: backward pass and backprop
        self.g_opt, self.d_opt = model_opt(g_loss=self.g_loss, d_loss=self.d_loss, learning_rate=learning_rate)

In [44]:
from collections import deque

class Memory():    
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size)
        self.states = deque(maxlen=max_size)

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [45]:
print('state size:{}'.format(states.shape), 
      'actions:{}'.format(actions.shape)) 
print('action size:{}'.format(np.max(actions) - np.min(actions)+1))

state size:(1111, 4) actions:(1111,)
action size:2


In [46]:
# Training parameters
# Network parameters
state_size = 4                 # number of units for the input state/observation -- simulation
action_size = 2                # number of units for the output actions -- simulation
hidden_size = 64               # number of units in each Q-network hidden layer -- simulation
batch_size = 32                # number of samples in the memory/ experience as mini-batch size
learning_rate = 0.001          # learning rate for adam

In [47]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, hidden_size=hidden_size, state_size=state_size, learning_rate=learning_rate)

# Init the memory
memory = Memory(max_size=batch_size)

(?, 4) (?, 64)
(1, ?, 64) (1, 64)
(1, ?, 64) (1, 64)
(?, 64)
(?, 2)
(?, 4) (?, 64)
(1, ?, 64) (1, 64)
(1, ?, 64) (1, 64)
(?, 64)
(?, 1)


In [48]:
state = env.reset()
for _ in range(batch_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    memory.buffer.append([state, action, next_state, reward, float(done)])
    state = next_state
    if done is True:
        state = env.reset()

In [49]:
memory.buffer[0]

[array([-0.01069676, -0.00322705,  0.03619071, -0.03418689]),
 0,
 array([-0.0107613 , -0.19884879,  0.03550697,  0.26969134]),
 1.0,
 0.0]

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
from collections import deque
episode_reward = deque(maxlen=100) # 100 episodes average/running average/running mean/window
saver = tf.train.Saver()
rewards_list, g_loss_list, d_loss_list = [], [], []
rates_list, d_lossQ_list, d_lossQsigm_list = [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Training episodes/epochs
    for ep in range(11111):
        batch = [] # every data batch
        total_reward = 0
        state = env.reset() # env first state
        g_initial_state = sess.run(model.g_initial_state)
        d_initial_state = sess.run(model.d_initial_state)

        # Training steps/batches
        while True:
            # Testing/inference
            action_logits, g_final_state, d_final_state = sess.run(
                fetches=[model.actions_logits, model.g_final_state, model.d_final_state], 
                feed_dict={model.states: np.reshape(state, [1, -1]),
                           model.g_initial_state: g_initial_state,
                           model.d_initial_state: d_initial_state})
            action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done)])
            memory.states.append([g_initial_state, g_final_state,
                                  d_initial_state, d_final_state])
            total_reward += reward
            g_initial_state = g_final_state
            d_initial_state = d_final_state
            state = next_state
            
            # Training
            batch = memory.buffer
            states = np.array([each[0] for each in batch])
            actions = np.array([each[1] for each in batch])
            next_states = np.array([each[2] for each in batch])
            rewards = np.array([each[3] for each in batch])
            dones = np.array([each[4] for each in batch])
            rnn_states = memory.states
            g_initial_states = np.array([each[0] for each in rnn_states])
            g_final_states = np.array([each[1] for each in rnn_states])
            d_initial_states = np.array([each[2] for each in rnn_states])
            d_final_states = np.array([each[3] for each in rnn_states])
            nextQs_logits = sess.run(fetches = model.Qs_logits,
                                     feed_dict = {model.states: next_states, 
                                                  model.g_initial_state: g_final_states[0].reshape([1, -1]),
                                                  model.d_initial_state: d_final_states[0].reshape([1, -1])})
            nextQs = nextQs_logits.reshape([-1]) * (1-dones) # exploit
            targetQs = rewards + (0.99 * nextQs)
            g_loss, d_loss, d_lossQ, d_lossQsigm, _, _ = sess.run(
                fetches=[model.g_loss, model.d_loss, 
                         model.d_lossQ, model.d_lossQ_sigm,
                         model.g_opt, model.d_opt], 
                feed_dict = {model.states: states, model.actions: actions,
                             model.targetQs: targetQs,
                             model.g_initial_state: g_initial_states[0].reshape([1, -1]),
                             model.d_initial_state: d_initial_states[0].reshape([1, -1])})

            if done is True:
                break

        # Episode total reward and success rate/prob
        episode_reward.append(total_reward) # stopping criteria
        rate = total_reward/ 500 # success is 500 points: 0-1
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'rate:{:.4f}'.format(rate),
              'gloss:{:.4f}'.format(g_loss),
              'dloss:{:.4f}'.format(d_loss),
              'dlossQ:{:.4f}'.format(d_lossQ),
              'dlossQsigm:{:.4f}'.format(d_lossQsigm))
        # Ploting out
        rewards_list.append([ep, np.mean(episode_reward)])
        rates_list.append([ep, rate])
        g_loss_list.append([ep, g_loss])
        d_loss_list.append([ep, d_loss])
        d_lossQ_list.append([ep, d_lossQ])
        d_lossQsigm_list.append([ep, d_lossQsigm])
        # Break episode/epoch loop
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:48.0000 rate:0.0960 gloss:0.5152 dloss:1.4261 dlossQ:0.9713 dlossQsigm:0.4548
Episode:1 meanR:54.0000 rate:0.1200 gloss:0.1059 dloss:3.5117 dlossQ:3.4335 dlossQsigm:0.0782
Episode:2 meanR:59.6667 rate:0.1420 gloss:0.0066 dloss:8.2521 dlossQ:8.1162 dlossQsigm:0.1359
Episode:3 meanR:60.7500 rate:0.1280 gloss:0.0627 dloss:9.2201 dlossQ:9.0784 dlossQsigm:0.1416
Episode:4 meanR:56.0000 rate:0.0740 gloss:0.2940 dloss:3.9147 dlossQ:3.8604 dlossQsigm:0.0543
Episode:5 meanR:62.0000 rate:0.1840 gloss:0.0487 dloss:18.0800 dlossQ:17.8812 dlossQsigm:0.1988
Episode:6 meanR:60.4286 rate:0.1020 gloss:0.1382 dloss:9.5126 dlossQ:9.3767 dlossQsigm:0.1359
Episode:7 meanR:59.5000 rate:0.1060 gloss:0.1514 dloss:5.9219 dlossQ:5.7190 dlossQsigm:0.2029
Episode:8 meanR:57.3333 rate:0.0800 gloss:-14.5985 dloss:17.7668 dlossQ:17.2592 dlossQsigm:0.5076
Episode:9 meanR:57.4000 rate:0.1160 gloss:0.0268 dloss:11.9522 dlossQ:11.5416 dlossQsigm:0.4106
Episode:10 meanR:71.7273 rate:0.4300 gloss:0.0039 dl

Episode:85 meanR:148.3256 rate:0.3600 gloss:0.0047 dloss:38.6580 dlossQ:38.3657 dlossQsigm:0.2922
Episode:86 meanR:152.3678 rate:1.0000 gloss:1.2108 dloss:256.4392 dlossQ:255.6707 dlossQsigm:0.7685
Episode:87 meanR:156.3182 rate:1.0000 gloss:0.4264 dloss:252.4097 dlossQ:251.6474 dlossQsigm:0.7624
Episode:88 meanR:157.3708 rate:0.5000 gloss:1.0918 dloss:138.2171 dlossQ:137.6527 dlossQsigm:0.5644
Episode:89 meanR:161.1778 rate:1.0000 gloss:0.0297 dloss:254.4239 dlossQ:253.6584 dlossQsigm:0.7654
Episode:90 meanR:164.9011 rate:1.0000 gloss:1.4100 dloss:197.7912 dlossQ:197.1163 dlossQsigm:0.6749
Episode:91 meanR:168.5435 rate:1.0000 gloss:1.1271 dloss:235.4438 dlossQ:234.7076 dlossQsigm:0.7363
Episode:92 meanR:172.1075 rate:1.0000 gloss:0.1981 dloss:241.4702 dlossQ:240.7243 dlossQsigm:0.7459
Episode:93 meanR:172.5957 rate:0.4360 gloss:1.3309 dloss:91.2905 dlossQ:90.8382 dlossQsigm:0.4523
Episode:94 meanR:174.5684 rate:0.7200 gloss:5.1505 dloss:185.7843 dlossQ:185.1308 dlossQsigm:0.6535
Epis

Episode:169 meanR:200.6100 rate:0.3380 gloss:0.0000 dloss:8.8003 dlossQ:8.6855 dlossQsigm:0.1148
Episode:170 meanR:203.4400 rate:1.0000 gloss:0.2001 dloss:240.8342 dlossQ:240.0894 dlossQsigm:0.7448
Episode:171 meanR:202.5600 rate:0.2240 gloss:0.0003 dloss:53.1096 dlossQ:52.9648 dlossQsigm:0.1448
Episode:172 meanR:202.5000 rate:0.2500 gloss:0.0003 dloss:38.9590 dlossQ:38.8438 dlossQsigm:0.1153
Episode:173 meanR:205.6800 rate:1.0000 gloss:0.4299 dloss:183.6344 dlossQ:182.9998 dlossQsigm:0.6346
Episode:174 meanR:208.9000 rate:1.0000 gloss:0.0000 dloss:181.7921 dlossQ:181.1445 dlossQsigm:0.6477
Episode:175 meanR:207.7100 rate:0.2740 gloss:0.0000 dloss:179.5241 dlossQ:179.1520 dlossQsigm:0.3720
Episode:176 meanR:206.5000 rate:0.2980 gloss:0.0033 dloss:94.3131 dlossQ:93.8826 dlossQsigm:0.4304
Episode:177 meanR:205.7900 rate:0.3720 gloss:0.0000 dloss:61.4041 dlossQ:61.0289 dlossQsigm:0.3752
Episode:178 meanR:208.7100 rate:1.0000 gloss:0.1163 dloss:232.9000 dlossQ:232.1681 dlossQsigm:0.7319
Ep

Episode:251 meanR:343.5800 rate:0.3740 gloss:0.0004 dloss:192.5699 dlossQ:191.9044 dlossQsigm:0.6654
Episode:252 meanR:347.6600 rate:1.0000 gloss:0.1996 dloss:199.9042 dlossQ:199.2254 dlossQsigm:0.6788
Episode:253 meanR:348.4300 rate:0.3420 gloss:0.0000 dloss:160.6407 dlossQ:160.0313 dlossQsigm:0.6094
Episode:254 meanR:352.2600 rate:1.0000 gloss:0.0000 dloss:191.3875 dlossQ:190.7227 dlossQsigm:0.6648
Episode:255 meanR:353.6600 rate:0.4740 gloss:0.0000 dloss:189.1950 dlossQ:188.5349 dlossQsigm:0.6602
Episode:256 meanR:354.2800 rate:0.3500 gloss:0.0000 dloss:169.7405 dlossQ:169.1141 dlossQsigm:0.6263
Episode:257 meanR:357.4400 rate:0.8800 gloss:0.0000 dloss:188.2865 dlossQ:187.6269 dlossQsigm:0.6595
Episode:258 meanR:357.9200 rate:0.3240 gloss:0.0000 dloss:173.3106 dlossQ:172.6776 dlossQsigm:0.6330
Episode:259 meanR:358.3200 rate:0.3320 gloss:0.0000 dloss:172.0706 dlossQ:171.4398 dlossQsigm:0.6307
Episode:260 meanR:358.4800 rate:0.3400 gloss:0.0005 dloss:162.8668 dlossQ:162.2529 dlossQsi

Episode:333 meanR:254.6300 rate:0.5160 gloss:0.0000 dloss:128.2273 dlossQ:127.6820 dlossQsigm:0.5453
Episode:334 meanR:250.9100 rate:0.2560 gloss:0.0003 dloss:123.1977 dlossQ:122.6631 dlossQsigm:0.5346
Episode:335 meanR:247.2100 rate:0.2600 gloss:0.0000 dloss:124.0608 dlossQ:123.5243 dlossQsigm:0.5365
Episode:336 meanR:243.4600 rate:0.2500 gloss:0.0000 dloss:119.2823 dlossQ:118.7561 dlossQsigm:0.5262
Episode:337 meanR:239.6500 rate:0.2380 gloss:0.0000 dloss:114.9189 dlossQ:114.4023 dlossQsigm:0.5166
Episode:338 meanR:238.9900 rate:0.2240 gloss:0.0000 dloss:110.3908 dlossQ:109.8844 dlossQsigm:0.5064
Episode:339 meanR:235.3300 rate:0.2680 gloss:0.0002 dloss:109.7796 dlossQ:109.2749 dlossQsigm:0.5047
Episode:340 meanR:231.6200 rate:0.2580 gloss:0.0000 dloss:109.0042 dlossQ:108.5009 dlossQsigm:0.5033
Episode:341 meanR:227.6900 rate:0.2140 gloss:0.0000 dloss:105.1620 dlossQ:104.6676 dlossQsigm:0.4944
Episode:342 meanR:227.5000 rate:0.3900 gloss:0.0001 dloss:113.1517 dlossQ:112.6395 dlossQsi

...

Episode:4068 meanR:178.6200 rate:0.2980 gloss:0.0033 dloss:133.0725 dlossQ:132.5181 dlossQsigm:0.5544
Episode:4069 meanR:182.3600 rate:1.0000 gloss:1.0792 dloss:174.8328 dlossQ:174.2050 dlossQsigm:0.6278
Episode:4070 meanR:183.2200 rate:0.3800 gloss:0.0000 dloss:163.1479 dlossQ:162.5335 dlossQsigm:0.6144
Episode:4071 meanR:183.2400 rate:0.2080 gloss:0.0000 dloss:148.0821 dlossQ:147.4965 dlossQsigm:0.5856
Episode:4072 meanR:180.6600 rate:0.4040 gloss:0.0000 dloss:147.7327 dlossQ:147.1478 dlossQsigm:0.5849
Episode:4073 meanR:180.7000 rate:0.2400 gloss:0.0000 dloss:138.0995 dlossQ:137.5338 dlossQsigm:0.5657
Episode:4074 meanR:177.7000 rate:0.3720 gloss:0.0000 dloss:135.6838 dlossQ:135.1230 dlossQsigm:0.5608
Episode:4075 meanR:177.6500 rate:0.2600 gloss:0.0000 dloss:129.7739 dlossQ:129.2253 dlossQsigm:0.5486
Episode:4076 meanR:177.5400 rate:0.2460 gloss:0.0000 dloss:124.2176 dlossQ:123.6807 dlossQsigm:0.5369
Episode:4077 meanR:177.8100 rate:0.2980 gloss:0.0000 dloss:122.6905 dlossQ:122.156

Episode:4149 meanR:183.2800 rate:0.2540 gloss:0.0000 dloss:148.3274 dlossQ:147.7422 dlossQsigm:0.5852
Episode:4150 meanR:183.0000 rate:0.2480 gloss:0.0002 dloss:139.2208 dlossQ:138.6532 dlossQsigm:0.5677
Episode:4151 meanR:182.9000 rate:0.2640 gloss:0.0000 dloss:136.2389 dlossQ:135.6777 dlossQsigm:0.5612
Episode:4152 meanR:186.1900 rate:1.0000 gloss:0.0034 dloss:178.9307 dlossQ:178.2876 dlossQsigm:0.6431
Episode:4153 meanR:182.7600 rate:0.2620 gloss:0.0000 dloss:163.3451 dlossQ:162.7303 dlossQsigm:0.6148
Episode:4154 meanR:182.6400 rate:0.2580 gloss:0.0000 dloss:151.5000 dlossQ:150.9077 dlossQsigm:0.5923
Episode:4155 meanR:182.5400 rate:0.2700 gloss:0.0000 dloss:143.0741 dlossQ:142.4984 dlossQsigm:0.5757
Episode:4156 meanR:186.3700 rate:1.0000 gloss:0.0041 dloss:165.7257 dlossQ:165.1065 dlossQsigm:0.6192
Episode:4157 meanR:186.4100 rate:0.2360 gloss:0.0000 dloss:151.4966 dlossQ:150.9043 dlossQsigm:0.5923
Episode:4158 meanR:186.5300 rate:0.2960 gloss:0.0000 dloss:144.4398 dlossQ:143.861

Episode:4230 meanR:184.3400 rate:0.3320 gloss:0.0035 dloss:116.4673 dlossQ:115.9550 dlossQsigm:0.5122
Episode:4231 meanR:183.9600 rate:0.2280 gloss:0.0000 dloss:109.2561 dlossQ:108.7522 dlossQsigm:0.5038
Episode:4232 meanR:183.6000 rate:0.2500 gloss:0.0000 dloss:107.5009 dlossQ:107.0011 dlossQsigm:0.4998
Episode:4233 meanR:183.4600 rate:0.2540 gloss:0.0000 dloss:106.3673 dlossQ:105.8701 dlossQsigm:0.4972
Episode:4234 meanR:183.7300 rate:0.3360 gloss:0.0000 dloss:110.6562 dlossQ:110.1492 dlossQsigm:0.5070
Episode:4235 meanR:183.9200 rate:0.2580 gloss:0.0000 dloss:109.1199 dlossQ:108.6164 dlossQsigm:0.5035
Episode:4236 meanR:184.4900 rate:0.3460 gloss:0.0000 dloss:113.5387 dlossQ:113.0252 dlossQsigm:0.5135
Episode:4237 meanR:184.5900 rate:0.2620 gloss:0.0000 dloss:111.7248 dlossQ:111.2153 dlossQsigm:0.5094
Episode:4238 meanR:184.6700 rate:0.2740 gloss:0.0000 dloss:110.9974 dlossQ:110.4896 dlossQsigm:0.5078
Episode:4239 meanR:184.6100 rate:0.2360 gloss:0.0001 dloss:108.4216 dlossQ:107.919

Episode:4311 meanR:167.0300 rate:1.0000 gloss:0.0244 dloss:173.0642 dlossQ:172.4318 dlossQsigm:0.6324
Episode:4312 meanR:167.4700 rate:0.3520 gloss:0.0000 dloss:163.6401 dlossQ:163.0248 dlossQsigm:0.6153
Episode:4313 meanR:164.1100 rate:0.3280 gloss:0.0000 dloss:155.3098 dlossQ:154.7102 dlossQsigm:0.5996
Episode:4314 meanR:163.9900 rate:0.3100 gloss:0.0000 dloss:148.0216 dlossQ:147.4361 dlossQsigm:0.5855
Episode:4315 meanR:164.5400 rate:0.3780 gloss:0.0001 dloss:146.1943 dlossQ:145.6124 dlossQsigm:0.5819
Episode:4316 meanR:164.7700 rate:0.3000 gloss:0.0000 dloss:140.3355 dlossQ:139.7652 dlossQsigm:0.5703
Episode:4317 meanR:164.9700 rate:0.3320 gloss:0.0000 dloss:137.5255 dlossQ:136.9610 dlossQsigm:0.5646
Episode:4318 meanR:165.3200 rate:0.3100 gloss:0.0000 dloss:134.0764 dlossQ:133.5188 dlossQsigm:0.5575
Episode:4319 meanR:165.7000 rate:0.3560 gloss:0.0000 dloss:134.0527 dlossQ:133.4952 dlossQsigm:0.5575
Episode:4320 meanR:166.1900 rate:0.3300 gloss:0.0000 dloss:132.4512 dlossQ:131.897

Episode:4392 meanR:231.9100 rate:0.3700 gloss:0.0000 dloss:159.7269 dlossQ:159.1189 dlossQsigm:0.6080
Episode:4393 meanR:232.7400 rate:0.4480 gloss:0.0000 dloss:154.6649 dlossQ:154.0668 dlossQsigm:0.5980
Episode:4394 meanR:234.2000 rate:0.5940 gloss:0.0000 dloss:162.9869 dlossQ:162.3731 dlossQsigm:0.6138
Episode:4395 meanR:237.8000 rate:1.0000 gloss:0.0000 dloss:191.0771 dlossQ:190.4127 dlossQsigm:0.6644
Episode:4396 meanR:241.1500 rate:1.0000 gloss:0.0002 dloss:213.2925 dlossQ:212.5909 dlossQsigm:0.7016
Episode:4397 meanR:241.2700 rate:0.3700 gloss:0.0000 dloss:189.8962 dlossQ:189.2339 dlossQsigm:0.6623
Episode:4398 meanR:241.6100 rate:0.3840 gloss:0.0000 dloss:181.9598 dlossQ:181.3113 dlossQsigm:0.6485
Episode:4399 meanR:245.0100 rate:1.0000 gloss:0.0003 dloss:211.6141 dlossQ:210.9153 dlossQsigm:0.6988
Episode:4400 meanR:248.5000 rate:1.0000 gloss:0.0122 dloss:205.8357 dlossQ:205.1474 dlossQsigm:0.6883
Episode:4401 meanR:251.4800 rate:1.0000 gloss:0.0000 dloss:216.0257 dlossQ:215.319

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(g_loss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('G losses')

In [ ]:
eps, arr = np.array(d_loss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('D losses')

In [ ]:
eps, arr = np.array(d_lossR_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('D losses R')

In [ ]:
eps, arr = np.array(d_lossQ_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('D losses Q')

## Testing

Let's checkout how our trained agent plays the game.

In [34]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')
# env = gym.make('Acrobot-v1')
# env = gym.make('MountainCar-v0')
# env = gym.make('Pendulum-v0')
# env = gym.make('Blackjack-v0')
# env = gym.make('FrozenLake-v0')
# env = gym.make('AirRaid-ram-v0')
# env = gym.make('AirRaid-v0')
# env = gym.make('BipedalWalker-v2')
# env = gym.make('Copy-v0')
# env = gym.make('CarRacing-v0')
# env = gym.make('Ant-v2') #mujoco
# env = gym.make('FetchPickAndPlace-v1') # mujoco required!

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(100):
    #while True:
        state = env.reset()
        total_reward = 0

        # Steps/batches
        #for _ in range(111111111111111111):
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: np.reshape(state, [1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                break
        # Print and break condition
        print('total_reward: {}'.format(total_reward))
        # if total_reward == 500:
        #     break
                
# Closing the env
env.close()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt


/home/arasdar/anaconda3/envs/env/lib/python3.6/site-packages/gym/__init__.py:22: UserWarning: DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.
  warnings.warn('DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.')


total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0


## Extending this

So, Cart-Pole is a pretty simple game. However, the same model can be used to train an agent to play something much more complicated like Pong or Space Invaders. Instead of a state like we're using here though, you'd want to use convolutional layers to get the state from the screen images.

![Deep Q-Learning Atari](assets/atari-network.png)

I'll leave it as a challenge for you to use deep Q-learning to train an agent to play Atari games. Here's the original paper which will get you started: http://www.davidqiu.com:8888/research/nature14236.pdf.